In [152]:
# Import Dependencies
import pandas as pd 
import json
import requests
import numpy as np
import datetime
from datetime import timedelta


In [153]:
# Read the CSV file from AWS to Pandas Dataframe
url = "https://finalproject-3.s3.us-west-1.amazonaws.com/2017.csv"

df = pd.read_csv(url)

df.head()

FL_DATE OP_CARRIER  ...  LATE_AIRCRAFT_DELAY Unnamed: 27
0  2017-01-01         AA  ...                  0.0         NaN
1  2017-01-01         AA  ...                  0.0         NaN
2  2017-01-01         AA  ...                 35.0         NaN
3  2017-01-01         AA  ...                  0.0         NaN
4  2017-01-01         AA  ...                  0.0         NaN

[5 rows x 28 columns]

In [222]:
# Filter the data for Tucson Airport
df1 = df.loc[df['ORIGIN'] == 'TUS']



# df1.to_csv('tus.csv')
# files.download('tus.csv')

# Remove the Unnamed columns which as no values
tus_df = df1.drop(['Unnamed: 27'], axis = 1)


In [223]:
# Fill all NaN with 0
from google.colab import files
tus_df1 = tus_df.fillna(0)



In [224]:
# if dep_time = 0 the File was cancelled

tus_df1['CRS_DEP_TIME'] = pd.to_datetime(tus_df1['CRS_DEP_TIME'], format = '%H%M').dt.time
tus_df1['CRS_ARR_TIME'] = pd.to_datetime(tus_df1['CRS_ARR_TIME'], format = '%H%M').dt.time

tus_df1
# tus_df1.to_csv('tus.csv')
# files.download('tus.csv')

FL_DATE OP_CARRIER  ...  SECURITY_DELAY LATE_AIRCRAFT_DELAY
73       2017-01-01         AA  ...             0.0                 0.0
206      2017-01-01         AA  ...             0.0                 0.0
323      2017-01-01         AA  ...             0.0                 0.0
851      2017-01-01         EV  ...             0.0                 0.0
1227     2017-01-01         OO  ...             0.0                 0.0
...             ...        ...  ...             ...                 ...
5672833  2017-12-31         EV  ...             0.0                 0.0
5673130  2017-12-31         EV  ...             0.0                 0.0
5673149  2017-12-31         EV  ...             0.0                 0.0
5673413  2017-12-31         UA  ...             0.0                 0.0
5674286  2017-12-31         UA  ...             0.0                 0.0

[15406 rows x 27 columns]

In [235]:
# Add Day of the week

tus_df1['FL_DATE'] = pd.to_datetime(tus_df1['FL_DATE'], format = '%Y/%m/%d')
tus_df1['day_of_week'] = tus_df1['FL_DATE'].dt.day_name()
tus_df1

# tus_df1.to_csv('tus.csv')
# files.download('tus.csv')



FL_DATE OP_CARRIER  ...  LATE_AIRCRAFT_DELAY day_of_week
73      2017-01-01         AA  ...                  0.0      Sunday
206     2017-01-01         AA  ...                  0.0      Sunday
323     2017-01-01         AA  ...                  0.0      Sunday
851     2017-01-01         EV  ...                  0.0      Sunday
1227    2017-01-01         OO  ...                  0.0      Sunday
...            ...        ...  ...                  ...         ...
5672833 2017-12-31         EV  ...                  0.0      Sunday
5673130 2017-12-31         EV  ...                  0.0      Sunday
5673149 2017-12-31         EV  ...                  0.0      Sunday
5673413 2017-12-31         UA  ...                  0.0      Sunday
5674286 2017-12-31         UA  ...                  0.0      Sunday

[15406 rows x 28 columns]

In [232]:
# Airport Codes
url_airport = "https://finalproject-3.s3.us-west-1.amazonaws.com/airports_codes.csv"

airport = pd.read_csv(url_airport)

airport.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9235 entries, 0 to 9234
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   AirportCode    9235 non-null   object
 1   CityName       9235 non-null   object
 2   WorldAreaCode  9235 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 216.6+ KB


In [236]:
merge_dest_df = pd.merge(airport, tus_df1, left_on='AirportCode', right_on='DEST', how='inner')

In [239]:
merge_dest_origin_df = pd.merge(airport, merge_dest_df, left_on='AirportCode', right_on='ORIGIN', how='inner')
merge_dest_origin_df

AirportCode_x  ... day_of_week
0               TUS  ...      Sunday
1               TUS  ...      Monday
2               TUS  ...      Monday
3               TUS  ...     Tuesday
4               TUS  ...     Tuesday
...             ...  ...         ...
15401           TUS  ...      Friday
15402           TUS  ...    Saturday
15403           TUS  ...    Saturday
15404           TUS  ...      Sunday
15405           TUS  ...      Sunday

[15406 rows x 34 columns]

In [244]:
# Rename Columns
rename_dest_origin_df = merge_dest_origin_df.rename(columns = {'CityName_x': 'origin_city_name',
                             'CityName_y': 'dest_city_name'})

rename_dest_origin_df

AirportCode_x  ... day_of_week
0               TUS  ...      Sunday
1               TUS  ...      Monday
2               TUS  ...      Monday
3               TUS  ...     Tuesday
4               TUS  ...     Tuesday
...             ...  ...         ...
15401           TUS  ...      Friday
15402           TUS  ...    Saturday
15403           TUS  ...    Saturday
15404           TUS  ...      Sunday
15405           TUS  ...      Sunday

[15406 rows x 34 columns]

In [250]:
# Drop colums not needed.
tus_final_df = rename_dest_origin_df.drop(['AirportCode_x', 'WorldAreaCode_x', 'AirportCode_y', 'WorldAreaCode_y'], axis = 1)
tus_final_df

# Export the final DF
tus_final_df.to_csv('tus_final_df.csv')
files.download('tus_final_df.csv')

tus_final_df



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

origin_city  ... day_of_week
0      TUCSON, ARIZONA, USA                 ...      Sunday
1      TUCSON, ARIZONA, USA                 ...      Monday
2      TUCSON, ARIZONA, USA                 ...      Monday
3      TUCSON, ARIZONA, USA                 ...     Tuesday
4      TUCSON, ARIZONA, USA                 ...     Tuesday
...                                    ...  ...         ...
15401  TUCSON, ARIZONA, USA                 ...      Friday
15402  TUCSON, ARIZONA, USA                 ...    Saturday
15403  TUCSON, ARIZONA, USA                 ...    Saturday
15404  TUCSON, ARIZONA, USA                 ...      Sunday
15405  TUCSON, ARIZONA, USA                 ...      Sunday

[15406 rows x 30 columns]